In [ ]:
%load_ext autoreload
%autoreload 2

In [19]:
import pandas as pd 
import os 
import numpy as np
import torch 


from law import ScalingLaw, MultiObjScalingLaw

# Functions

In [20]:
def load_break_losses(dirs, slice_list, task_name):
    files_to_exclude = ['trace', 'val_inputs', 'labels', 'proportions', 'emb', 'rouge', 'generations', 'gradient', 'acc', 'matrices']
    df_all = pd.DataFrame() 

    for dir in dirs:
        files = os.listdir(dir)
        files = [os.path.join(dir, f) for f in files]
        files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

        for file in files:
            if ".log" in file or task_name not in file:
                continue 

            if any(skill not in file for skill in slice_list):
                continue 

            if "break" not in file:
                continue

            break_steps = int(file.split("break_")[-1].split("_")[0])
           
            method = file.split("/")[-1]

            if "doremi" not in file:
                continue 

            runs = os.listdir(file)
            for run in runs:

                if "test_" in run:
                    continue

                if any([exclude_file in run for exclude_file in files_to_exclude]):
                    continue 

                seed = int(run.split("seed_")[-1].split("_")[0])
                checkpoint = int(run.split("-")[-1].split(".")[0])


                path = os.path.join(file, run)

                df = pd.read_pickle(path)

                df = df.rename(columns={"task_idx": "skill", "task_loss": "loss"})

                df["method"] = method
                df["seed"] = seed
                df["checkpoint"] = checkpoint
                df["break_steps"] = break_steps


                df.set_index("checkpoint", inplace=True)


                df_all = pd.concat([df_all, df])


    df_all = df_all.sort_values(by=["checkpoint", "seed"])
    return df_all


In [21]:
def load_resume_losses(dirs, slice_list, task_name):
    files_to_exclude = ['trace', 'val_inputs', 'labels', 'proportions', 'emb', 'rouge', 'generations', 'gradient', 'acc', 'matrices']    
    df_all = pd.DataFrame() 

    for dir in dirs:
        files = os.listdir(dir)
        files = [os.path.join(dir, f) for f in files] 
        files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

        for file in files:
            if ".log" in file or task_name not in file:
                continue 

            if any(skill not in file for skill in slice_list):
                continue 

            if "resume" not in file:
                continue

            if "doremi" not in file:
                continue 


            method = file.split("/")[-1]

            new_weight_str = method.split("weights_")[-1].split("_")[0]
            new_a, new_b, new_c = [float(f"0.{weight}") for weight in new_weight_str.split("0.")[1:]]


            break_steps = int(file.split(f"resume_doremi_")[-1].split("_")[0])


            runs = os.listdir(file)
            for run in runs:

                if "test_" in run:
                    continue

                if any([exclude_file in run for exclude_file in files_to_exclude]):
                    continue 

                seed = int(run.split("seed_")[-1].split("_")[0])
                checkpoint = int(run.split("-")[-1].split(".")[0])


                path = os.path.join(file, run)

                df = pd.read_pickle(path)

                df = df.rename(columns={"task_idx": "skill", "task_loss": "loss"})

                df["method"] = method
                df["seed"] = seed
                df["checkpoint"] = checkpoint
                df["break_steps"] = break_steps
                df["new_p1"] = new_a 
                df["new_p2"] = new_b 
                df["new_p3"] = new_c 

                df.set_index("checkpoint", inplace=True)


                df_all = pd.concat([df_all, df])


    df_all = df_all.sort_values(by=["checkpoint", "new_p1", "seed"])
    return df_all


In [22]:
def load_doremi_matrices(dirs, slice_list, break_steps):
    matrices = {}

    slice_str = "_".join(slice_list)
    for dir in dirs:
        files = os.listdir(dir)
        files = [os.path.join(dir, f) for f in files]

        for file in files:
            
            if "break" in file and f"stratified_{slice_str}_doremi" in file:

                s = int(file.split("_break_")[-1].split("_")[0])
                if s != break_steps:
                    continue
                runs = os.listdir(file)
                for run in runs:
                    if "avg" not in run:
                        continue
                    if "drm_matrices.npy" not in run:
                        continue 
                    path = os.path.join(file, run)
                    print(path)
                    seed = int(run.split("seed_")[-1].split("_")[0])

                    

                    A = np.load(path)
                    matrices[seed] = A.mean(axis=0)

    return matrices

In [23]:
def calculate_r_squared(actuals, predictions):
    actuals, predictions = actuals.numpy(), predictions.numpy()
    # Calculate the total sum of squares
    total_sum_of_squares = np.sum((actuals - np.mean(actuals)) ** 2)
    # Calculate the residual sum of squares
    residual_sum_of_squares = np.sum((actuals - predictions) ** 2)
    # Calculate R-squared
    r_squared = 1 - (residual_sum_of_squares / total_sum_of_squares)
    return r_squared


In [25]:
def mixing_law_full(x, param):
    # one set of params per skill
    #print(param)

    result = torch.matmul(x, param)
    return result

def init_params_law_full(idx, num_domains=3):
    #for c_i in np.linspace(0.5, 5, 10):
    for _ in range(30):
        ts = [-np.random.rand() if i == idx else np.random.rand() * 0.1 for i in range(num_domains)]
        yield ts

In [27]:
def make_individual_xy_full(df_break, df_resume, skill, break_steps, seed):
    x = []
    y = []

    df_break_subset = df_break[(df_break.break_steps == break_steps) & (df_break.seed == seed) & (df_break.skill == skill)]
    df_break_subset = df_break_subset.loc[df_break_subset.index.max()]

    df_resume_subset = df_resume[(df_resume.break_steps == break_steps) & (df_resume.seed == seed) & (df_resume.skill == skill)]
    df_resume_subset = df_resume_subset.loc[df_resume_subset.index.max()]
    
        
    loss_0 = df_break_subset.loss
    x = df_resume_subset[['new_p1', 'new_p2', 'new_p3']].values
    y = df_resume_subset['loss'].values - loss_0

    return x, y

In [65]:
def make_xy_joint(df_break, df_resume, matrix, break_steps, seed):
    x = []
    y = []

    df_break_subset = df_break[(df_break.break_steps == break_steps) & (df_break.seed == seed)]
    df_break_subset = df_break_subset.loc[df_break_subset.index.max()]

    df_resume_subset = df_resume[(df_resume.break_steps == break_steps) & (df_resume.seed == seed)]
    df_resume_subset = df_resume_subset.loc[df_resume_subset.index.max()]

    x = df_resume_subset[['new_p1_normalized', 'new_p2_normalized', 'new_p3_normalized']].drop_duplicates(keep='first').values.T
    x = matrix.dot(x)

    y = df_resume_subset['loss'].values.reshape(-1, 3) - df_break_subset.loss.values

    return x, y

In [67]:
def law_1(x, param):
    b = param[0]
    return b*x[0]

def law_2(x, param):
    b = param[0]
    return b*x[1]

def law_3(x, param):
    b = param[0]
    return b*x[2]


def param_generator_joint():
    for b in np.linspace(-10, 0, 5):
        yield [b]

# Arxiv Book Stackexchange

In [49]:
dirs = ["../output/09252024/", "../output/09262024/"] # REPLACE WITH YOUR RUN OUTPUT DIRECTORIES
task_name = "slimpj"
slice_list = ['arxiv', 'book', 'stackexchange']
df_break = load_break_losses(dirs, slice_list, task_name)

In [50]:
skills = sorted(df_break.skill.unique())

In [51]:
df_resume = load_resume_losses(dirs, skills, task_name)
df_resume['new_p1_normalized'] = df_resume.apply(lambda x: x['new_p1']/(x['new_p1'] + x['new_p2'] + x['new_p3']), axis=1)
df_resume['new_p2_normalized'] = df_resume.apply(lambda x: x['new_p2']/(x['new_p1'] + x['new_p2'] + x['new_p3']), axis=1)
df_resume['new_p3_normalized'] = df_resume.apply(lambda x: x['new_p3']/(x['new_p1'] + x['new_p2'] + x['new_p3']), axis=1)

In [52]:
df_resume = df_resume[df_resume.index==1500]

In [53]:
break_steps = sorted(df_resume.break_steps.unique())
seeds = sorted(df_break.seed.unique())

break_steps = [1000]

## Get $A^{t \star}$ 

In [ ]:
params = {skill : {bs: {seed: {} for seed in seeds} for bs in break_steps} for skill in skills }

x_per_skill = {skill : {bs: {seed: {} for seed in seeds} for bs in break_steps} for skill in skills }
y_per_skill = {skill : {bs: {seed: {} for seed in seeds} for bs in break_steps} for skill in skills }

mses = []
r2s = []
for i, skill in enumerate(skills):
    for bs in break_steps:
        for seed in seeds:
            print(skill, bs, seed)
            x, y = make_individual_xy_full(df_break, df_resume, skill, bs, seed)
        
            x_per_skill[skill][bs][seed] = x
            y_per_skill[skill][bs][seed] = y
                        
            law = ScalingLaw(mixing_law_full)
            p = law.fit(x, y, init_params_law_full(i, num_domains=len(skills)), max_step=100, delta=0.02)
            params[skill][bs][seed] = p # param

            prediction_train = mixing_law_full(torch.tensor(x, dtype=torch.float), torch.tensor(p, dtype=torch.float))
            mse_train = torch.nn.functional.mse_loss(prediction_train, torch.tensor(y, dtype=torch.float)).item()
            r2_train = calculate_r_squared(torch.tensor(y), torch.tensor(prediction_train))

            mses.append(mse_train)
            r2s.append(r2_train)


            print(f"MSE: {mse_train}, R2: {r2_train}")


mses = np.array(mses)
r2s = np.array(r2s)

print(mses.mean(), mses.std())
print(r2s.mean(), r2s.std())


In [38]:
import pickle 

with open("./law_results/arxiv_books_stackexchange/params_doremi_trajectory_opt_1000_steps_500.pkl", "wb") as f:
    pickle.dump(params, f)

## Get $\tilde{A}^t = b^t A^t$

In [ ]:
doremi_matrices = load_doremi_matrices(dirs, slice_list, 1000)

In [ ]:
params_doremi = {bs: {seed: {} for seed in seeds} for bs in break_steps}


x_per_skill_doremi = {bs: {seed: {} for seed in seeds} for bs in break_steps}
y_per_skill_doremi = {bs: {seed: {} for seed in seeds} for bs in break_steps}

for bs in break_steps:
    for seed in seeds:
        print(bs, seed)
        x, ys = make_xy_joint(df_break, df_resume, doremi_matrices[seed], bs, seed)
        
        x_per_skill_doremi[bs][seed] = x
        y_per_skill_doremi[bs][seed] = ys
                    
        law = MultiObjScalingLaw([law_1, law_2, law_3])
        p = law.fit(x, ys.T, param_generator_joint(), max_step=100, delta=0.02)
        params_doremi[bs][seed] = p

In [70]:
import pickle 

with open("./law_results/arxiv_books_stackexchange/params_doremi_trajectory_doremi_matrix_1000_steps_500.pkl", "wb") as f:
    pickle.dump(params_doremi, f)